In [15]:
import math

class CurrentGenerator:
    def __init__(self):
        self.source=[]
        
    def createbase(self):
        self.base_tok=((math.sin(i/180*math.pi))*50 for i in range(0,3600,18))
        for i in self.base_tok:
            print('mmm',i)
            yield i
            
    def create1stLevel(self,g):
        self.base_tok=((math.sin(i/180*math.pi))*50 for i in range(0,720,18))
        for i in self.base_tok:
            yield i
            
        self.tok1stLevel1=((math.sin(i/180*math.pi))*50*g for i in range(720,36000,18))
        for i in self.tok1stLevel1:
            yield i
    

            
class CircuitBreaker:
    def __init__(self,name):
        self.name=name        
        self.__state=False
        
    def turn_breaker_on(self):
        self.__state=True
    
    def turn_breaker_off(self):
        self.__state=False
    
    def get_state(self):
        return self.__state
        
class Izmeritel:
    def __init__(self,name,cb,generator):
        self.name=name
        self.cb=cb
        self.generator=generator
        
    def get_data(self):
        if self.cb.get_state():
            return next(self.generator)
        else:
            return 0
        
        
class Mtz:
    def __init__(self,name,tt,cb):
        self.stepName=name
        self.measurment=[]
        self.tt=tt
        self.cb=cb
    
    def tick(self,st_list):
        for i in range(20):
            self.measurment.append(self.tt.get_data())
        print('Рассмотри следующий период синусоиды') #,self.measurment)
        while len(self.measurment)>20:
            self.measurment.pop(0)
        self.run(st_list)


    def run(self,st_list):
        i_sq=self.calc_sq(self.measurment)
        print('Действующее значение периода %.2f' %(i_sq))
        for st in st_list:
            if i_sq>st.ustavka1:
                st.counter1+=1
            else:
                st.counter1=1
            if st.counter1>st.step:
                self.trip(st.stepName)  #срабатывание    
                
            
    def trip(self,name):
        self.cb.turn_breaker_off()
        print()
        print('!!! Выключатель отключен ступенью - ', name,' !!!')
        
        
    def calc_sq(self,value):
        return sum(((m*m)**(1/2))/len(value) for m in value)  #sqrt(sum....)
    
class Stupeny:
    def __init__(self,name,ustavka1,step):
        self.stepName=name
        self.ustavka1=ustavka1
        self.step=step
        self.counter1=1

In [16]:
st_list=[]
cb1 = CircuitBreaker("CB1")
cb1.turn_breaker_on()
st1=Stupeny('3st',34,10)
st2=Stupeny('2nd',70,6)
st3=Stupeny('1rd',100,3)
st_list.append(st1)
st_list.append(st2)
st_list.append(st3)


i=1

print(cb1.name)

current_generator=CurrentGenerator()

for i in range(2,5):
    print()
    print()
    print('Синусоида номер ',i-1)
    current_generator.source=current_generator.create1stLevel(i)


    tt1=Izmeritel("TT1",cb1,current_generator.source)
    mtz_1st=Mtz('Mrz',tt1,cb1)
    print('Состояние выключателя', cb1.get_state())
    print()
    
    while cb1.get_state(): #пока выключатель работает
        mtz_1st.tick(st_list)
        print()
    
    print('Состояние выключателя', cb1.get_state())
    print()
    
    cb1.turn_breaker_on()

CB1


Синусоида номер  1
Состояние выключателя True

Рассмотри следующий период синусоиды
Действующее значение периода 31.57

Рассмотри следующий период синусоиды
Действующее значение периода 31.57

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

Рассмотри следующий период синусоиды
Действующее значение периода 63.14

!!! Выключатель отключен ступенью -  3st  !!!

Состояние выключателя Fa